In [ ]:
# %% Deep learning - Section 7.47
#    ANN for classifying querties

# This code pertains a deep learning course provided by Mike X. Cohen on Udemy:
#   > https://www.udemy.com/course/deeplearning_x
# The "base" code in this repository is adapted (with very minor modifications)
# from code developed by the course instructor (Mike X. Cohen), while the
# "exercises" and the "code challenges" contain more original solutions and
# creative input from my side. If you are interested in DL (and if you are
# reading this statement, chances are that you are), go check out the course, it
# is singularly good.


In [ ]:
# %% Libraries and modules
import numpy               as np
import matplotlib.pyplot   as plt
import torch
import torch.nn            as nn
import copy

from google.colab                     import files
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')


In [ ]:
# %% Data

nClust = 100
blur   = 1

A = [1,1]
B = [5,1]

a = [ A[0]+np.random.randn(nClust)*blur, A[1]+np.random.randn(nClust)*blur ]
b = [ B[0]+np.random.randn(nClust)*blur, B[1]+np.random.randn(nClust)*blur ]

# True labels
labels_np = np.vstack(( np.zeros((nClust,1)), np.ones((nClust,1)) ))

# Concatenate
data_np = np.hstack((a,b)).T

# Convert into torch tensor
data   = torch.tensor(data_np).float()
labels = torch.tensor(labels_np).float()


In [ ]:
# Plotting

fig = plt.figure(figsize=(7,7))

plt.plot( data[np.where(labels==0)[0],0],data[np.where(labels==0)[0],1],'bs' )
plt.plot( data[np.where(labels==1)[0],0],data[np.where(labels==1)[0],1],'ro' )
plt.title('Some binary data')
plt.xlabel('x1')
plt.ylabel('x2')

plt.savefig('figure15_ann_binary_classification.png')

plt.show()

files.download('figure15_ann_binary_classification.png')


In [ ]:
# %% Inspect data types

print(type(data_np))
print(np.shape(data_np))
print( )

print(type(data))
print(np.shape(data))


In [ ]:
# %% Build the model

ANNclassify = nn.Sequential(
                 nn.Linear(2,1),   # input layer (num inputs, num outputs)
                 nn.ReLU(),        # first activation function
                 nn.Linear(1,1),   # output layer (num inputs, num outputs)
                 nn.Sigmoid()      # second activation function
                 )

ANNclassify


In [ ]:
# %% Training parameters

# Learning rate
learning_rate = 0.01

# Loss function (binary cross-entropy for binary category data)
loss_fun = nn.BCELoss()

# Optimizer (i.e. the flavour of gradient to implement; here stocastic gradient descent)
optimizer = torch.optim.SGD(ANNclassify.parameters(),lr=learning_rate)


In [ ]:
# %% Train the model

# Parameters
num_epochs = 1000
losses     = torch.zeros(num_epochs)

## Training
for epoch_i in range(num_epochs):

    # Forward propagation
    yHat = ANNclassify(data)

    # Loss
    loss = loss_fun(yHat,labels)
    losses[epoch_i] = loss

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


In [ ]:
# %% Show losses
# Notice that when the losses do not plateau, it suggests that the model can still learn

# Plotting
plt.plot(losses.detach(),'o',markerfacecolor='w',linewidth=.1)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title(f'Final loss = {np.round(losses.detach()[-1].item(),4)}')

plt.savefig('figure16_ann_binary_classification.png')

plt.show()

files.download('figure16_ann_binary_classification.png')


In [ ]:
# %% Compute predictions

# Final forward pass
predictions = ANNclassify(data)
pred_labels = predictions > 0.5

# Find errors
misclassified = np.where(pred_labels != labels)[0]

# Total accuracy
tot_acc = 100 - 100*len(misclassified)/(2*nClust)

print(f'Final accuracy = {tot_acc}%')


In [ ]:
# %% Plot labelled data

fig = plt.figure(figsize=(7,7))

plt.plot( data[misclassified,0],data[misclassified,1],'gx',markersize=12,markeredgewidth=3 )
plt.plot( data[np.where(~pred_labels)[0],0],data[np.where(~pred_labels)[0],1],'bs' )
plt.plot( data[np.where(pred_labels)[0],0],data[np.where(pred_labels)[0],1] ,'ro' )
plt.xlabel('x1')
plt.ylabel('x2')
plt.legend(['Misclassified','blue','red'])
plt.title(f'Classification accuracy = {tot_acc}%')

plt.savefig('figure17_ann_binary_classification.png')

plt.show()

files.download('figure17_ann_binary_classification.png')


In [ ]:
# %% Exercise 1
#    It is intuitive that the model can reach 100% accuracy if the qwerties are more separable. Modify the
#    qwerty-generating code to get the model to have 100% classification accuracy.

# E.g., modify the location of the centroid B to B = [8,3]. Surprisingly, one datapoint still gets
# mislabelled despite the clusters being so obviously divided! In such a scenario even a simple KNN
# or a K-mean clustering algorithm would perform flawlessy


In [ ]:
# %% Exercise 2
#    It is common in DL to train the model for a specified number of epochs. But you can also train until
#    the model reaches a certain accuracy criterion. Re-write the code so that the model continues training
#    until it reaches 90% accuracy.
#    What would happen if the model falls into a local minimum and never reaches 90% accuracy? Yikes! You can
#    force-quit a process in google-colab by clicking on the top-left 'play' button of a code cell.

# Generate data and then run this code:

# %% Build the model
ANNclassify = nn.Sequential(
                 nn.Linear(2,1),   # input layer (num inputs, num outputs)
                 nn.ReLU(),        # first activation function
                 nn.Linear(1,1),   # output layer (num inputs, num outputs)
                 nn.Sigmoid()      # second activation function
                 )

# %% Training parameters
learning_rate = 0.01
loss_fun = nn.BCELoss()
optimizer = torch.optim.SGD(ANNclassify.parameters(),lr=learning_rate)

# %% Train the model (max epoch to avoid infinite loops)
max_epochs = 5000
losses     = []
acc_thresh = 0.9
epoch_i    = 0

while epoch_i < max_epochs:

    yHat = ANNclassify(data)

    loss = loss_fun(yHat,labels)
    losses.append(loss.item())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    predictions = ANNclassify(data)
    pred_labels = predictions > 0.5

    tot_acc = (pred_labels == labels).float().mean().item()

    if tot_acc >= acc_thresh:
        print(f'Stopping training at epoch {epoch_i} with accuracy {tot_acc:.4f}')
        break

    epoch_i += 1


In [ ]:
# %%  Plotting

plt.plot(losses,'o',markerfacecolor='w',linewidth=.1)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title(f'Final loss = {np.round(losses[-1],4)}')

plt.savefig('figure22_ann_binary_classification_extra2.png')

plt.show()

files.download('figure22_ann_binary_classification_extra2.png')


In [ ]:
# %% Plotting

misclassified = np.where(pred_labels != labels)[0]

fig = plt.figure(figsize=(7,7))

plt.plot( data[misclassified,0],data[misclassified,1],'gx',markersize=12,markeredgewidth=3 )
plt.plot( data[np.where(~pred_labels)[0],0],data[np.where(~pred_labels)[0],1],'bs' )
plt.plot( data[np.where(pred_labels)[0],0],data[np.where(pred_labels)[0],1] ,'ro' )
plt.xlabel('x1')
plt.ylabel('x2')
plt.legend(['Misclassified','blue','red'])
plt.title(f'Classification accuracy = {np.round(tot_acc,2)}%')

plt.savefig('figure23_ann_binary_classification_extra2.png')

plt.show()

files.download('figure23_ann_binary_classification_extra2.png')
